In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

# Load features
path = '/content/drive/MyDrive/Watashara_Projects/TIP/'
import pandas as pd

w2vec = pd.read_csv(path + 'features/word2vec_TIP_all.csv')
esm2 = pd.read_csv(path + 'features/Esm2_TIP_all.csv', header=None).iloc[:, :]
prot_t5_bfd = pd.read_csv(path + 'features/TIP_al_prot_t5_xl_bfd.csv', header=None).iloc[1:, 1:]
prot_t5_unief50 = pd.read_csv(path + 'features/TIP_all_prot_t5_xl_uniref50.csv', header=None).iloc[1:, 1:]
fasttext = pd.read_csv(path + 'features/Fasttext_features_TIP.csv').values
bert = pd.read_csv(path + 'features/bert_features_TIP.csv').values
FEGS = pd.read_csv(path + 'features/FEGS_features.csv', header=None).iloc[1:, :].values
reducedACID_all = pd.read_csv(path + 'features/reducedACID_all.csv', header=None).iloc[1:, 1:]
reducedCHARGE_all = pd.read_csv(path + 'features/reducedCHARGE_all.csv', header=None).iloc[1:, 1:]
AAC_all = pd.read_csv(path + 'features/AAC_all.csv', header=None).iloc[1:, 1:]
PAAC_all = pd.read_csv(path + 'features/PAAC_all.csv', header=None).iloc[1:, 1:]
Multi_headed_fused_features = pd.read_csv(path + 'features/Multi_headed_fused_features.csv')


# Combine all features
# all_features = np.column_stack((w2vec, esm2, prot_t5_unief50,fasttext,bert,FEGS))
all_features = np.column_stack((w2vec, fasttext,bert, esm2,prot_t5_bfd, prot_t5_unief50,FEGS,reducedACID_all,reducedCHARGE_all,AAC_all,PAAC_all))



print(np.shape(w2vec))
print(np.shape(esm2))
print(np.shape(prot_t5_bfd))
print(np.shape(prot_t5_unief50))
print(np.shape(fasttext))
print(np.shape(bert))
print(np.shape(FEGS))
print(np.shape(reducedCHARGE_all))
print(np.shape(reducedACID_all))
print(np.shape(AAC_all))
print(np.shape(PAAC_all))
print(np.shape(Multi_headed_fused_features))
print(np.shape(all_features))



(708, 120)
(708, 320)
(708, 1024)
(708, 1024)
(708, 120)
(708, 768)
(708, 578)
(708, 32)
(708, 32)
(708, 20)
(708, 21)
(708, 3035)
(708, 4059)


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

all_feat = np.array(all_features)
# Standardize the data
scaler = StandardScaler()
data_scaled = scaler.fit_transform(all_features)


df_feat = pd.DataFrame(data_scaled)
print(df_feat.shape)
# Store the original indices
original_indices = list(range(df_feat.shape[1]))

# Drop features with 0 standard deviation
zero_std_features = df_feat.columns[df_feat.std() == 0].tolist()
df_feat = df_feat.loc[:, df_feat.std() != 0]
remaining_indices_after_zero_std = df_feat.columns.tolist()

# Mapping to original indices
remaining_indices_after_zero_std = [original_indices[i] for i in remaining_indices_after_zero_std]

# Features dropped due to zero standard deviation
dropped_zero_std = [i for i in original_indices if i not in remaining_indices_after_zero_std]

# Drop features with high correlation
threshold = 0.8
cor = df_feat.corr().abs()
print(df_feat.shape)
# Create a mask to identify pairs with correlation greater than the threshold
high_corr_var = np.where(cor > threshold)
high_corr_var = [(cor.columns[x], cor.columns[y]) for x, y in zip(*high_corr_var) if x != y and x < y]

# Track dropped features due to high correlation
to_drop = set()
for pair in high_corr_var:
    to_drop.add(pair[1])

df_feat = df_feat.drop(columns=to_drop)
print(df_feat.shape)
remaining_indices_after_high_corr = df_feat.columns.tolist()

# Mapping to original indices
final_indices = remaining_indices_after_high_corr

# Features dropped due to high correlation
dropped_high_corr = [i for i in remaining_indices_after_zero_std if i not in final_indices]

# Save the final features to CSV with original indices
final_features = df_feat.values
final_df = pd.DataFrame(final_features, columns=final_indices)
final_df.to_csv(path + 'Features_selection/allfeat_TIP_CF.csv', index=False)

# Save dropped indices information
drop_info = {
    "dropped_due_to_zero_std": dropped_zero_std,
    "dropped_due_to_high_corr": dropped_high_corr,
    "final_retained_indices": final_indices
}
drop_info_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in drop_info.items()]))
drop_info_df.to_csv(path + 'Features_selection/dropped_allfeatures_info.csv', index=False)

print("Features dropped due to zero standard deviation: ", dropped_zero_std)
print("Features dropped due to high correlation: ", dropped_high_corr)
print("Final retained feature indices: ", final_indices)

(708, 4059)
(708, 3885)
(708, 3542)
Features dropped due to zero standard deviation:  [3376, 3377, 3378, 3379, 3380, 3381, 3382, 3383, 3384, 3385, 3386, 3387, 3388, 3389, 3390, 3391, 3392, 3393, 3394, 3395, 3396, 3397, 3398, 3399, 3400, 3401, 3402, 3403, 3404, 3405, 3406, 3407, 3408, 3409, 3410, 3411, 3412, 3413, 3414, 3415, 3416, 3417, 3418, 3419, 3420, 3421, 3422, 3423, 3424, 3425, 3426, 3427, 3428, 3429, 3430, 3431, 3432, 3433, 3434, 3435, 3436, 3437, 3438, 3439, 3440, 3441, 3442, 3443, 3444, 3445, 3446, 3447, 3448, 3449, 3450, 3451, 3452, 3453, 3454, 3455, 3456, 3457, 3458, 3459, 3460, 3461, 3462, 3463, 3464, 3465, 3466, 3467, 3468, 3469, 3470, 3471, 3472, 3473, 3474, 3475, 3476, 3477, 3478, 3479, 3480, 3481, 3482, 3483, 3484, 3485, 3486, 3487, 3488, 3489, 3490, 3491, 3492, 3493, 3494, 3495, 3496, 3497, 3498, 3499, 3500, 3501, 3502, 3503, 3504, 3505, 3506, 3507, 3508, 3509, 3510, 3511, 3512, 3513, 3514, 3515, 3516, 3517, 3518, 3519, 3520, 3521, 3522, 3523, 3524, 3525, 3526, 3527, 3

In [ ]:
!pip install lightning
!pip install sklearn-contrib-lightning
!pip install torch_geometric
!pip dgl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.0/811.0 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.2/869.2 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 56.0 MB/s eta 0:00:00
ERROR: unknown command "dgl"


In [ ]:
# -*- coding: utf-8 -*-

import os
import numpy as np
from sklearn.preprocessing import scale,StandardScaler
from sklearn.preprocessing import normalize,Normalizer
from sklearn.preprocessing import minmax_scale, MinMaxScaler
from sklearn.preprocessing import maxabs_scale, MaxAbsScaler
from sklearn.decomposition import KernelPCA, SparsePCA,PCA
from sklearn.decomposition import FactorAnalysis
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso, LassoCV
from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LassoLarsCV, LassoLars
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC,SVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.manifold import SpectralEmbedding
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.linear_model import OrthogonalMatchingPursuitCV
from lightning.classification import CDClassifier

def zscore_scaler(data):
    data=scale(data)
    return data
def normalizer(data):
    data = Normalizer().fit_transform(data)
    return data
def minmaxscaler(data):
    data=MinMaxScaler().fit_transform(data)
    return data
def maxabsscaler(data):
    data=MaxAbsScaler().fit_transform(data)
    return data
def zeroMean(dataMat):
    meanVal=np.mean(dataMat,axis=0)#axis represents the obtained mean value by column
    stdVal=np.std(dataMat,axis=0)
    newData=dataMat-meanVal
    new_data=newData/stdVal
    return new_data,meanVal
def covArray(dataMat):
    #obtain the  covariance matrix
    covMat=np.cov(dataMat,rowvar=0)
    return covMat
def featureMatrix(dataMat):
    eigVals,eigVects=np.linalg.eig(np.mat(dataMat))
	#datermine the eigenvalue and eigenvector
    return eigVals,eigVects
def percentage2n(eigVals,percentage=0.99):
    #percentage represents the rate of contribution
    sortArray=np.sort(eigVals)   #ascending sort
    sortArray=sortArray[-1::-1]  #descending order
    arraySum=sum(sortArray)
    tmpSum=0
    num=0
    for i in sortArray:
        tmpSum+=i
        num+=1
        if tmpSum>=arraySum*percentage:
            return num #num is the number of remaining principal component

def lassodimension(data,label,alpha=np.array([0.01,0.05,0.1])):#The alpha value range is 0.001, 0.002, 0.005, 0.01, 0.015, 0.02
    lassocv=LassoCV(cv=5, alphas=alpha).fit(data, label)
    x_lasso = lassocv.fit(data,label)#Substituting alpha for dimensionality reduction
    mask = x_lasso.coef_ != 0
    new_data = data[:,mask]
    return new_data,mask
# using factor analysis to reduce the dimension
def factorAnalysis(data,percentage = 0.9):
    dataMat = np.array(data)
    newData,meanVal=zeroMean(data)  #equalization
    covMat=covArray(newData)  #covariance matrix
    eigVals,eigVects=featureMatrix(covMat)
    n_components = percentage2n(eigVals,percentage)
    clf = FactorAnalysis(n_components=n_components)
    new_data = clf.fit_transform(dataMat)
    return new_data  # return the new data
# using principal component to reduce dimension
def pca(data,percentage = 0.9):
    dataMat = np.array(data)
    newData,meanVal=zeroMean(data)  #equalization
    covMat=covArray(newData)  #covariance matrix
    eigVals,eigVects=featureMatrix(covMat)
    n_components = percentage2n(eigVals,percentage)
    clf=PCA(n_components=n_components)
    new_x = clf.fit_transform(dataMat)
    return new_x
#using kernel principal component to reduce dimension
def KPCA(data,percentage=0.9):
    dataMat = np.array(data)
    newData,meanVal=zeroMean(data)
    covMat=covArray(newData)
    eigVals,eigVects=featureMatrix(covMat)
    n_components = percentage2n(eigVals,percentage)
    #n_component=n_components
    clf=KernelPCA(n_components=n_components,kernel='rbf',gamma=1/1318)#rbf linear poly
    new_x=clf.fit_transform(dataMat)
    return new_x
def SPCA(data,percentage=0.9):
    dataMat = np.array(data)
    newData,meanVal=zeroMean(data)
    covMat=covArray(newData)
    eigVals,eigVects=featureMatrix(covMat)
    n_components = percentage2n(eigVals,percentage)
    #n_component=n_components
    clf=SparsePCA(n_components=n_components)#rbf linear poly
    new_x=clf.fit_transform(dataMat)
    return new_x
def SE(data,n_components=300):
    embedding = SpectralEmbedding(n_components=n_components)
    X_transformed = embedding.fit_transform(data)
    return X_transformed
def LLE(data,n_components=300):
    embedding = LocallyLinearEmbedding(n_components=n_components)
    X_transformed = embedding.fit_transform(data)
    return X_transformed
#Lasso中的正交匹配追踪
def omp_omp(data,label,n_nonzero_coefs=300):
    omp = OrthogonalMatchingPursuit(n_nonzero_coefs=n_nonzero_coefs)
    omp.fit(data, label)
    coef = omp.coef_
    idx_r, = coef.nonzero()
    #omp_cv = OrthogonalMatchingPursuitCV()
    #omp_cv.fit(X, y_noisy)
    #coef = omp_cv.coef_
    #idx_r, = coef.nonzero()
    new_data=data[:,idx_r]
    return new_data,idx_r
#卡方进行特征选择
def chi2_chi2(data,label,k=300):
    model_chi2= SelectKBest(chi2, k=k)
    new_data=model_chi2.fit_transform(data,label)
    return new_data
#SVC降维
def selectFromLinearSVC(data,label,lamda):
    lsvc = LinearSVC(C=lamda, penalty="l1", dual=False).fit(data,label)
    model = SelectFromModel(lsvc,prefit=True)
    new_data= model.transform(data)
    return new_data
def TSVD(data,n_components=500):
    svd = TruncatedSVD(n_components=n_components)
    new_data=svd.fit_transform(data)
    return new_data

def Light_lasso(X,y,alpha_):
     clf = CDClassifier(penalty="l1/l2",
                   loss="squared_hinge",
                   #multiclass=True,
                   max_iter=50,
                   alpha=alpha_,
                   C=1.0 / X.shape[0],
                   tol=1e-3)
     clf.fit(X, y  )
     H1,H2=np.nonzero(clf.coef_)
     X=X[:,H2]
     return X,H2

In [ ]:
import scipy.io as sio
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import scale,StandardScaler



data_=pd.read_csv(path + 'Features_selection/allfeat_TIP_CF.csv')
data=np.array(data_)
data=data[:,:]
[m1,n1]=np.shape(data)
label1=np.ones((206,1))#Value can be changed
label2=np.zeros((502,1))
labels=np.append(label1,label2)
shu=scale(data)
X=shu
y=labels
ata_2,importance=Light_lasso(X,y.T.ravel(),0.03)
data_2,importance=Light_lasso(X,y,0.03)
shu=data_2
data_csv = pd.DataFrame(data=shu)
data_csv.to_csv(path + 'Features_selection/GL_allCF.csv',index=False)
data_csv = pd.DataFrame(data=importance)
data_csv.to_csv(path + 'Features_selection/GL_allCF_importance.csv')


RP

In [ ]:
import numpy as np
from sklearn import random_projection
from sklearn.preprocessing import scale
import pandas as pd

data_=pd.read_csv(path + 'Features_selection/allfeat_TIP_CF.csv')
data=np.array(data_)
data=data[:,:]
[m1,n1]=np.shape(data)
label1=np.ones((206,1))#Value can be changed
label2=np.zeros((502,1))
labels=np.append(label1,label2)
shu=scale(data)
X=shu
y=labels
rp = random_projection.SparseRandomProjection(n_components=300, random_state=66)
X_projected = rp.fit_transform(shu)
shu=X_projected
data_csv = pd.DataFrame(data=shu)
data_csv.to_csv(path + 'Features_selection/RP_alltrain.csv')


**TSVD**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import scale,StandardScaler
from sklearn.decomposition import TruncatedSVD
# from sklearn.random_projection import sparse_random_matrix
from scipy.sparse import random as sparse_random_matrix
import scipy.io as sio



data_=pd.read_csv(path + 'Features_selection/allfeat_TIP_CF.csv')
data=np.array(data_)
data=data[:,:]
[m1,n1]=np.shape(data)
label1=np.ones((206,1))#Value can be changed
label2=np.zeros((502,1))
labels=np.append(label1,label2)
shu=scale(data)
X=data
y=labels
svd = TruncatedSVD(n_components=500, n_iter=10, random_state=42)
hist=svd.fit(X)
new_data=svd.transform(X)
data_csv = pd.DataFrame(data=new_data)
data_csv.to_csv(path + 'Features_selection/TSVD_allfeat_TIP.csv',index=False)

**Information gain**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import scale
from sklearn.manifold import SpectralEmbedding

def SE_select(data,n_components=300):
    embedding = SpectralEmbedding(n_components=n_components)
    X_transformed = embedding.fit_transform(data)
    return X_transformed

data_ = pd.read_csv(path + 'Features_selection/allfeat_TIP_CF.csv')
data = np.array(data_)
[m1, n1] = np.shape(data)

label1 = np.ones((206, 1))  # Adjust values as needed
label2 = np.zeros((502, 1))
label = np.append(label1, label2)

Zongshu = scale(data)
RNA_shu = Zongshu[:,0:72]
pro_shu = Zongshu[:,72:]

new_RNA_data = SE_select(RNA_shu, n_components=7)
new_pro_data = SE_select(pro_shu, n_components=30)

data_new = np.hstack((new_RNA_data,new_pro_data))
optimal_RPI_features = pd.DataFrame(data=data_new)
optimal_RPI_features.to_csv(path + 'Features_selection/SE_allfeat_TIP.csv', index=False)

In [ ]:
!pip install dgl -f https://data.dgl.ai/wheels/repo.html




Looking in links: https://data.dgl.ai/wheels/repo.html
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 27.0 MB/s eta 0:00:00


**GAE**

In [ ]:
from sklearn.neighbors import NearestNeighbors
import torch
from torch_geometric.nn import GCNConv, GAE
import pandas as pd
import numpy as np
import networkx as nx
from torch_geometric.data import Data

# Load data and create labels
data_ = pd.read_csv(path + 'Features_selection/allfeat_TIP_CF.csv')
data = np.array(data_)
[m1, n1] = np.shape(data)
label1 = np.ones((206, 1))  # Value can be changed
label2 = np.zeros((502, 1))
labels = np.append(label1, label2)

# Apply scaling if needed
# shu = scale(data)
X = data
y = labels

# Nearest neighbors graph construction
k = 5
nbrs = NearestNeighbors(n_neighbors=k, metric='euclidean').fit(data)
distances, indices = nbrs.kneighbors(data)

graph = nx.Graph()
graph.add_nodes_from(range(len(data)))

for i in range(len(data)):
    for j in indices[i]:
        if i != j:
            graph.add_edge(i, j)

# Create edge_index for PyTorch Geometric
edge_index = torch.LongTensor(list(graph.edges)).t().contiguous()

# Update num_features to match actual feature size
num_features = data.shape[1]  # This should be 3167 as per the error message

data = Data(x=torch.tensor(data, dtype=torch.float32), edge_index=edge_index)

class GCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True)
        self.conv2 = GCNConv(2 * out_channels, out_channels, cached=True)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

# Parameters
out_channels = 1200
epochs = 180

# Initialize GAE model with correct feature size
model = GAE(GCNEncoder(num_features, out_channels))

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(data.x, data.edge_index)
    loss = model.recon_loss(z, data.edge_index)
    loss.backward()
    optimizer.step()
    return float(loss)

# Training loop
for epoch in range(epochs):
    loss = train()
    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss}')

# Save the embeddings
Z = model.encode(data.x, data.edge_index).detach().numpy()
df = pd.DataFrame(Z)
csv_file_path = ''
df.to_csv(path + 'Features_selection/GAE_allfeat_TIP_1200.csv', index=False)


Epoch [1/180], Loss: 34.53878402709961
Epoch [2/180], Loss: 34.53878402709961
Epoch [3/180], Loss: 34.53878402709961
Epoch [4/180], Loss: 34.53878402709961
Epoch [5/180], Loss: 34.53878402709961
Epoch [6/180], Loss: 34.53878402709961
Epoch [7/180], Loss: 34.53878402709961
Epoch [8/180], Loss: 34.53878402709961
Epoch [9/180], Loss: 34.53878402709961
Epoch [10/180], Loss: 34.53878402709961
Epoch [11/180], Loss: 34.53878402709961
Epoch [12/180], Loss: 34.53878402709961
Epoch [13/180], Loss: 34.53878402709961
Epoch [14/180], Loss: 34.53878402709961
Epoch [15/180], Loss: 34.53878402709961
Epoch [16/180], Loss: 34.53878402709961
Epoch [17/180], Loss: 34.53878402709961
Epoch [18/180], Loss: 34.53878402709961
Epoch [19/180], Loss: 34.53878402709961
Epoch [20/180], Loss: 34.53878402709961
Epoch [21/180], Loss: 34.53878402709961
Epoch [22/180], Loss: 34.53878402709961
Epoch [23/180], Loss: 34.53878402709961
Epoch [24/180], Loss: 34.53878402709961
Epoch [25/180], Loss: 34.53878402709961
Epoch [26

**PCA**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


# data = pd.read_csv(r'fusion_features\Data\fusion\All_features.csv', header=None, index_col=False)
path = "/content/drive/MyDrive/Watashara_Projects/TIP/"

data_ = pd.read_csv(path + 'Features_selection/allfeat_TIP_CF.csv')
data = np.array(data_)
[m1, n1] = np.shape(data)
label1 = np.ones((206, 1))  # Value can be changed
label2 = np.zeros((502, 1))
labels = np.append(label1, label2)

# Apply scaling if needed
# shu = scale(data)
X = data
y = labels

# scaler = StandardScaler()
# data_scaled = scaler.fit_transform(data)

#
pca = PCA(n_components=700)  #
principalComponents = pca.fit_transform(X)

#
print('Explained variance ratio:', pca.explained_variance_ratio_)

#
principalDf = pd.DataFrame(data=principalComponents)

#
principalDf.to_csv(path+f'Features_selection/PCA_TIP_800.csv', index=False, header=None)

ValueError: n_components=800 must be between 0 and min(n_samples, n_features)=708 with svd_solver='full'

**Elastic Net**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import ElasticNet, ElasticNetCV

# Path where your data is stored
path = "/content/drive/MyDrive/Watashara_Projects/TIP/"



data_ = pd.read_csv(path + 'Features_selection/allfeat_TIP_CF.csv')
data = np.array(data_)
[m1, n1] = np.shape(data)
label1 = np.ones((206, 1))  # Value can be changed
label2 = np.zeros((502, 1))
labels = np.append(label1, label2)

# Apply scaling if needed
# shu = scale(data)
X = data_
y = labels

print("Shape of X:", X.shape)
print("Shape of y:", y.shape)
print("Number of zeros in y:", np.count_nonzero(y == 0))
print("Number of ones in y:", np.count_nonzero(y == 1))

# Define ElasticNet with cross-validation
model_cv = ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1],
                        alphas=[0.01, 0.1, 1.0, 10.0], cv=10)

# Train the model using cross-validation
model_cv.fit(X, y)

# Output the best alpha and l1_ratio found
print('Best alpha: ', model_cv.alpha_)
print('Best l1_ratio: ', model_cv.l1_ratio_)

# Train the final model using the best parameters
model = ElasticNet(alpha=model_cv.alpha_, l1_ratio=model_cv.l1_ratio_)
model.fit(X, y)

# Get feature importance (coefficients)
importance = model.coef_

# Identify the indices of the selected features (non-zero coefficients)
selected_indices = np.where(importance != 0)[0]

# Create a DataFrame with feature importance and their original indices
importance_df = pd.DataFrame({
    'feature_index': selected_indices,
    'importance_score': importance[selected_indices]
})

# Save the importance scores and indices to a CSV file
importance_df.to_csv(path + 'Features_selection/feature_importance_EN_TIP.csv', index=False)

# Optionally, save the selected features (based on non-zero coefficients) to another CSV
selected_features_df = X.iloc[:, selected_indices]
selected_features_df.to_csv(path + 'Features_selection/EN_features_TIP.csv', index=False)


Shape of X: (708, 3542)
Shape of y: (708,)
Number of zeros in y: 502
Number of ones in y: 206


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.797e-02, tolerance: 1.064e-02
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.681e-02, tolerance: 1.064e-02
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.048e-02, tolerance: 1.103e

Best alpha:  0.1
Best l1_ratio:  0.1


In [3]:
!pip install mrmr-selection

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 6.8 MB/s eta 0:00:00


In [ ]:
# select top n features using mRMR
import numpy as np
from mrmr import mrmr_classif
import pandas as pd

# Path where your data is stored
path = "/content/drive/MyDrive/Watashara_Projects/TIP/"



data_ = pd.read_csv(path + 'Features_selection/allfeat_TIP_CF.csv')
data = np.array(data_)
[m1, n1] = np.shape(data)
label1 = np.ones((206, 1))  # Value can be changed
label2 = np.zeros((502, 1))
labels = np.append(label1, label2)

# Apply scaling if needed
# shu = scale(data)
X = data_
y = labels


X = pd.DataFrame(data_)
selected_features = mrmr_classif(X=X, y=labels, K=700)


X_selected = X[selected_features]


df = pd.DataFrame(X_selected)


df.to_csv(path + 'Features_selection/mRMR_700.csv', index=False, header=None)

100%|██████████| 700/700 [12:43<00:00,  1.09s/it]
